# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$ (see also Problem 1 in hw3t). In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in hw4-theoretical.

### Question 2

To implement your reduction from Problem 1 in hw4t, you will work with some standard benchmark instances for min-cost flow included in the hw4p folder on canvas. The format of the data is described in the Info file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):

# Read in the datafile
    with open(infile, 'r') as description:
        data = description.read()
    
    components = data.strip().splitlines()

    
# Split data into descriptions by component used to create graph
    comments = []
    problem = []
    node_desc = []
    arc_desc = []

    for i in components: 
        if i[0] == 'c':
            comments.append(i)
        elif i[0] == 'p':
            problem.append(i)
        elif i[0] == 'n':
            node_desc.append(i)
        elif i[0] == 'a':
            arc_desc.append(i)

            
# Begin to create the DiGraph
    graph = nx.DiGraph()

    
# Generate the nodes using problem, numbered 1 through n      
    num_nodes = int(problem[0].split()[2])
    graph.add_nodes_from(range(1, num_nodes + 1), demand = 0)

    
# Generate a list of nodes with demands, using node_desc, add them to the graph
    for i in node_desc:
        node_demands = i.split()
        n = int(node_demands[1])
        d = int(node_demands[2])
        graph.node[n]['demand'] = d

        
# Add edges from head to tail nodes, with capacities and weights as noted in the file, if an edge between the two does not already exist
# If an edge exists between the nodes, proceed to the else clause; add an intermediate node with a demand of 0

    for i in arc_desc:
        e = i.split()
        tail = int(e[1])
        head = int(e[2])
        cap = int(e[4])
        cost = int(e[5])
        if graph.has_edge(head, tail) == False:
            graph.add_edge(head, tail, capacity = cap, weight = cost)
        else:
            num_nodes = num_nodes + 1
            graph.add_node(num_nodes, demand = 0) # Add intermediate node with demand 0
            graph.add_edge(head, num_nodes, capacity = cap, weight = cost) # Add edge from head node to intermediate node, assign same capacity to maintain demand of 0 at this node
            graph.add_edge(num_nodes, tail, capacity = cap, weight = 0) # Add edge from intermediate node to tail node, assign same capacity to maintain demand of 0 at this node

    return graph


In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print("Correct value for _40 instance:", nx.flow.min_cost_flow_cost(G_40) == 52099553858)
print("Correct value for _6830 instance:", nx.flow.min_cost_flow_cost(G_6830) == 299390431788)
print("Correct value for _176280 instance:", nx.flow.min_cost_flow_cost(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    
    prob = pulp.LpProblem('LP_Flow_Value', pulp.LpMinimize)
    demand = nx.get_node_attributes(G, 'demand')
    cap = nx.get_edge_attributes(G, 'capacity')
    cost = nx.get_edge_attributes(G, 'weight')
    
    
# The variables- the flow on each edge
    flow = pulp.LpVariable.dicts('flow', G.edges(), 0, cat = 'Integer') 

    
# Objective Function- minimize flow cost
    cost_list = []
    for edge in G.edges():
        edge_cost = flow[edge] * cost[edge]
        cost_list.append(edge_cost)
        
    prob += sum(cost_list)


# Capacity constraint
    for edge in G.edges():
        prob += flow[edge] <= cap[edge]

        
# Demand constraint
    for node in G.nodes():
        flow_in = sum(flow[i] for i in G.in_edges(node))
        flow_out = sum(flow[o] for o in G.out_edges(node))
        prob += flow_in - flow_out == demand[node]
        
        
    prob.solve()
    
    solution = int(pulp.value(prob.objective))
    
    return solution


The following will check that the LP finds the same optimal values as previously.

In [4]:
print( "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858)
print( "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788)
print( "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
